In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs
import time

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)

ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

ivt_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]


ivt_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]

ivt_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]


ivt_list_merra2_polar=[]
ivt_max_list_merra2_polar=[]
iwv_list_merra2_polar=[]
iwv_max_list_merra2_polar=[]
s_list_merra2_polar=[]
s_max_list_merra2_polar=[]
mfc_list_merra2_polar=[]
mfc_max_list_merra2_polar=[]

ivt_list_merra2_fixed=[]
ivt_max_list_merra2_fixed=[]
iwv_list_merra2_fixed=[]
iwv_max_list_merra2_fixed=[]
s_list_merra2_fixed=[]
s_max_list_merra2_fixed=[]
mfc_list_merra2_fixed=[]
mfc_max_list_merra2_fixed=[]

ivt_list_merra2_fixed_poleward=[]
ivt_max_list_merra2_fixed_poleward=[]
iwv_list_merra2_fixed_poleward=[]
iwv_max_list_merra2_fixed_poleward=[]
s_list_merra2_fixed_poleward=[]
s_max_list_merra2_fixed_poleward=[]
mfc_list_merra2_fixed_poleward=[]
mfc_max_list_merra2_fixed_poleward=[]

num_test=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if year>=1980:
        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
        labeled=ds['ar_labeled']
        
        ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
        ivt_merra2=ds['ivt'][0::2,:,:]
        ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
        iwv_merra2=ds['iwv'][0::2,:,:]
        ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
        v=ds['v']
        s_merra2=np.sqrt(u**2+v**2)
        
        ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
        mfc_merra2=ds['mfc'][0::2,:,:]

        if i==0:
            weights=np.cos(np.deg2rad(ds.lat))
        
        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt_merra2[j,:,:]
            iwv_sel=iwv_merra2[j,:,:]
            s_sel=s_merra2[j,:,:]
            mfc_sel=mfc_merra2[j,:,:]

            num_day=0
            for k in range(len(unique)):
                
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()

                if area_feature>0:

                    ivt_sel1=np.where(sel_feature.values>0,ivt_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),ivt_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    ivt_sel1=dk['test']
                    
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=np.where(sel_feature.values>0,iwv_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),iwv_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    iwv_sel1=dk['test']
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=np.where(sel_feature.values>0,s_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),s_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    s_sel1=dk['test']
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=np.where(sel_feature.values>0,mfc_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),mfc_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    mfc_sel1=dk['test']
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(mfc_sel.lon.values,mfc_sel.lat.values,ivt_sel1,vmin=0,vmax=800,cmap=plt.cm.seismic)
                        plt.show()

                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(mfc_sel.lon.values,mfc_sel.lat.values,mfc_sel1,vmin=-20e-5,vmax=20e-5,cmap=plt.cm.seismic)
                        plt.show()

                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                    
                    num_day=num_day+1
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,ivt_sel,
                                     vmin=0,vmax=800,cmap=plt.cm.Greys,alpha=0.2)
                        ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,ivt_sel1,
                                     vmin=0,vmax=800,cmap=plt.cm.Greys)

                        print('IVT:'+str(ivt_mean)+','+str(ivt_max))
                        print('IWV:'+str(iwv_mean)+','+str(iwv_max))
                        print('S:'+str(s_mean)+','+str(s_max))
                        print('IVTDIV:'+str(ivtdiv_mean)+','+str(ivtdiv_max))
                        print('MFC:'+str(mfc_mean)+','+str(mfc_max))
                        plt.show()

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

            #sys.exit()
        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_merra2_polar.append(mean_ivt)
        ivt_max_list_merra2_polar.append(max_ivt)
        iwv_list_merra2_polar.append(mean_iwv)
        iwv_max_list_merra2_polar.append(max_iwv)
        s_list_merra2_polar.append(mean_s)
        s_max_list_merra2_polar.append(max_s)
        mfc_list_merra2_polar.append(mean_mfc)
        mfc_max_list_merra2_polar.append(max_mfc)
        
        
        num_test.append(len(ivt_list_year))
        
        for j in range(len(nums_array_year)):
            ivt_array_merra2_polar[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_merra2_polar[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_merra2_polar[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_merra2_polar[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_merra2_polar[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_merra2_polar[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_merra2_polar[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_merra2_polar[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('A')
        #################

        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled'][0::2,:,:]

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt_merra2[j,:,:]
            iwv_sel=iwv_merra2[j,:,:]
            s_sel=s_merra2[j,:,:]
            mfc_sel=mfc_merra2[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()

                if area_feature>0:
                    ivt_sel1=np.where(sel_feature.values>0,ivt_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),ivt_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    ivt_sel1=dk['test']
                    
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=np.where(sel_feature.values>0,iwv_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),iwv_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    iwv_sel1=dk['test']
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=np.where(sel_feature.values>0,s_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),s_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    s_sel1=dk['test']
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=np.where(sel_feature.values>0,mfc_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),mfc_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    mfc_sel1=dk['test']
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()
                    
                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_merra2_fixed.append(mean_ivt)
        ivt_max_list_merra2_fixed.append(max_ivt)
        iwv_list_merra2_fixed.append(mean_iwv)
        iwv_max_list_merra2_fixed.append(max_iwv)
        s_list_merra2_fixed.append(mean_s)
        s_max_list_merra2_fixed.append(max_s)
        mfc_list_merra2_fixed.append(mean_mfc)
        mfc_max_list_merra2_fixed.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_merra2_fixed[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_merra2_fixed[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_merra2_fixed[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_merra2_fixed[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_merra2_fixed[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_merra2_fixed[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_merra2_fixed[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_merra2_fixed[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('B')
        
        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
        labeled=ds['ar_labeled']

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt_merra2[j,:,:]
            iwv_sel=iwv_merra2[j,:,:]
            s_sel=s_merra2[j,:,:]
            mfc_sel=mfc_merra2[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()

                if area_feature>0:
                    
                    ivt_sel1=np.where(sel_feature.values>0,ivt_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),ivt_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    ivt_sel1=dk['test']
                    
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=np.where(sel_feature.values>0,iwv_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),iwv_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    iwv_sel1=dk['test']
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=np.where(sel_feature.values>0,s_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),s_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    s_sel1=dk['test']
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=np.where(sel_feature.values>0,mfc_sel.values,np.nan)
                    dk=xr.Dataset()
                    dk['test']=(('lat','lon'),mfc_sel1)
                    dk['lat']=sel_feature.lat
                    dk['lon']=sel_feature.lon
                    mfc_sel1=dk['test']
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_merra2_fixed_poleward.append(mean_ivt)
        ivt_max_list_merra2_fixed_poleward.append(max_ivt)
        iwv_list_merra2_fixed_poleward.append(mean_iwv)
        iwv_max_list_merra2_fixed_poleward.append(max_iwv)
        s_list_merra2_fixed_poleward.append(mean_s)
        s_max_list_merra2_fixed_poleward.append(max_s)
        mfc_list_merra2_fixed_poleward.append(mean_mfc)
        mfc_max_list_merra2_fixed_poleward.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_merra2_fixed_poleward[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_merra2_fixed_poleward[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_merra2_fixed_poleward[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_merra2_fixed_poleward[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_merra2_fixed_poleward[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_merra2_fixed_poleward[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array_merra2_fixed_poleward[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array_merra2_fixed_poleward[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))          
       
    print('C')
    print(num_test)

In [ ]:
dk=xr.Dataset()

dk['ivt_array_merra2_polar']=(('months','years'),np.asarray(ivt_array_merra2_polar))
dk['ivt_max_array_merra2_polar']=(('months','years'),np.asarray(ivt_max_array_merra2_polar))
dk['iwv_array_merra2_polar']=(('months','years'),np.asarray(iwv_array_merra2_polar))
dk['iwv_max_array_merra2_polar']=(('months','years'),np.asarray(iwv_max_array_merra2_polar))
dk['s_array_merra2_polar']=(('months','years'),np.asarray(s_array_merra2_polar))
dk['s_max_array_merra2_polar']=(('months','years'),np.asarray(s_max_array_merra2_polar))
dk['mfc_array_merra2_polar']=(('months','years'),np.asarray(mfc_array_merra2_polar))
dk['mfc_max_array_merra2_polar']=(('months','years'),np.asarray(mfc_max_array_merra2_polar))

dk['ivt_array_merra2_fixed']=(('months','years'),np.asarray(ivt_array_merra2_fixed))
dk['ivt_max_array_merra2_fixed']=(('months','years'),np.asarray(ivt_max_array_merra2_fixed))
dk['iwv_array_merra2_fixed']=(('months','years'),np.asarray(iwv_array_merra2_fixed))
dk['iwv_max_array_merra2_fixed']=(('months','years'),np.asarray(iwv_max_array_merra2_fixed))
dk['s_array_merra2_fixed']=(('months','years'),np.asarray(s_array_merra2_fixed))
dk['s_max_array_merra2_fixed']=(('months','years'),np.asarray(s_max_array_merra2_fixed))
dk['mfc_array_merra2_fixed']=(('months','years'),np.asarray(mfc_array_merra2_fixed))
dk['mfc_max_array_merra2_fixed']=(('months','years'),np.asarray(mfc_max_array_merra2_fixed))

dk['ivt_array_merra2_fixed_poleward']=(('months','years'),np.asarray(ivt_array_merra2_fixed_poleward))
dk['ivt_max_array_merra2_fixed_poleward']=(('months','years'),np.asarray(ivt_max_array_merra2_fixed_poleward))
dk['iwv_array_merra2_fixed_poleward']=(('months','years'),np.asarray(iwv_array_merra2_fixed_poleward))
dk['iwv_max_array_merra2_fixed_poleward']=(('months','years'),np.asarray(iwv_max_array_merra2_fixed_poleward))
dk['s_array_merra2_fixed_poleward']=(('months','years'),np.asarray(s_array_merra2_fixed_poleward))
dk['s_max_array_merra2_fixed_poleward']=(('months','years'),np.asarray(s_max_array_merra2_fixed_poleward))
dk['mfc_array_merra2_fixed_poleward']=(('months','years'),np.asarray(mfc_array_merra2_fixed_poleward))
dk['mfc_max_array_merra2_fixed_poleward']=(('months','years'),np.asarray(mfc_max_array_merra2_fixed_poleward))


dk['ivt_list_merra2_polar']=(('years'),ivt_list_merra2_polar)
dk['ivt_max_list_merra2_polar']=(('years'),ivt_max_list_merra2_polar)
dk['iwv_list_merra2_polar']=(('years'),iwv_list_merra2_polar)
dk['iwv_max_list_merra2_polar']=(('years'),iwv_max_list_merra2_polar)
dk['s_list_merra2_polar']=(('years'),s_list_merra2_polar)
dk['s_max_list_merra2_polar']=(('years'),s_max_list_merra2_polar)
dk['mfc_list_merra2_polar']=(('years'),mfc_list_merra2_polar)
dk['mfc_max_list_merra2_polar']=(('years'),mfc_max_list_merra2_polar)

dk['ivt_list_merra2_fixed']=(('years'),ivt_list_merra2_fixed)
dk['ivt_max_list_merra2_fixed']=(('years'),ivt_max_list_merra2_fixed)
dk['iwv_list_merra2_fixed']=(('years'),iwv_list_merra2_fixed)
dk['iwv_max_list_merra2_fixed']=(('years'),iwv_max_list_merra2_fixed)
dk['s_list_merra2_fixed']=(('years'),s_list_merra2_fixed)
dk['s_max_list_merra2_fixed']=(('years'),s_max_list_merra2_fixed)
dk['mfc_list_merra2_fixed']=(('years'),mfc_list_merra2_fixed)
dk['mfc_max_list_merra2_fixed']=(('years'),mfc_max_list_merra2_fixed)


dk['ivt_list_merra2_fixed_poleward']=(('years'),ivt_list_merra2_fixed_poleward)
dk['ivt_max_list_merra2_fixed_poleward']=(('years'),ivt_max_list_merra2_fixed_poleward)
dk['iwv_list_merra2_fixed_poleward']=(('years'),iwv_list_merra2_fixed_poleward)
dk['iwv_max_list_merra2_fixed_poleward']=(('years'),iwv_max_list_merra2_fixed_poleward)
dk['s_list_merra2_fixed_poleward']=(('years'),s_list_merra2_fixed_poleward)
dk['s_max_list_merra2_fixed_poleward']=(('years'),s_max_list_merra2_fixed_poleward)
dk['mfc_list_merra2_fixed_poleward']=(('years'),mfc_list_merra2_fixed_poleward)
dk['mfc_max_list_merra2_fixed_poleward']=(('years'),mfc_max_list_merra2_fixed_poleward)

dk.coords['years']=np.arange(1980,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_synoptic_merra2.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_synoptic_merra2.nc',mode='w',format='NETCDF4')

In [ ]:
dir_data='/Volumes/Extreme Pro/'
ds=xr.open_dataset(dir_data+'ar_chars_synoptic_merra2.nc')

ivt_array_merra2_polar=ds['ivt_array_merra2_polar'].values
ivt_max_array_merra2_polar=ds['ivt_max_array_merra2_polar'].values
iwv_array_merra2_polar=ds['iwv_array_merra2_polar'].values
iwv_max_array_merra2_polar=ds['iwv_max_array_merra2_polar'].values
s_array_merra2_polar=ds['s_array_merra2_polar'].values
s_max_array_merra2_polar=ds['s_max_array_merra2_polar'].values
mfc_array_merra2_polar=ds['mfc_array_merra2_polar'].values
mfc_max_array_merra2_polar=ds['mfc_max_array_merra2_polar'].values

ivt_array_merra2_fixed=ds['ivt_array_merra2_fixed'].values
ivt_max_array_merra2_fixed=ds['ivt_max_array_merra2_fixed'].values
iwv_array_merra2_fixed=ds['iwv_array_merra2_fixed'].values
iwv_max_array_merra2_fixed=ds['iwv_max_array_merra2_fixed'].values
s_array_merra2_fixed=ds['s_array_merra2_fixed'].values
s_max_array_merra2_fixed=ds['s_max_array_merra2_fixed'].values
mfc_array_merra2_fixed=ds['mfc_array_merra2_fixed'].values
mfc_max_array_merra2_fixed=ds['mfc_max_array_merra2_fixed'].values

ivt_list_merra2_fixed=ds['ivt_list_merra2_fixed'].values.tolist()
ivt_max_list_merra2_fixed=ds['ivt_max_list_merra2_fixed'].values.tolist()
iwv_list_merra2_fixed=ds['iwv_list_merra2_fixed'].values.tolist()
iwv_max_list_merra2_fixed=ds['iwv_max_list_merra2_fixed'].values.tolist()
s_list_merra2_fixed=ds['s_list_merra2_fixed'].values.tolist()
s_max_list_merra2_fixed=ds['s_max_list_merra2_fixed'].values.tolist()
mfc_list_merra2_fixed=ds['mfc_list_merra2_fixed'].values.tolist()
mfc_max_list_merra2_fixed=ds['mfc_max_list_merra2_fixed'].values.tolist()

ivt_list_merra2_polar=ds['ivt_list_merra2_polar'].values.tolist()
ivt_max_list_merra2_polar=ds['ivt_max_list_merra2_polar'].values.tolist()
iwv_list_merra2_polar=ds['iwv_list_merra2_polar'].values.tolist()
iwv_max_list_merra2_polar=ds['iwv_max_list_merra2_polar'].values.tolist()
s_list_merra2_polar=ds['s_list_merra2_polar'].values.tolist()
s_max_list_merra2_polar=ds['s_max_list_merra2_polar'].values.tolist()
mfc_list_merra2_polar=ds['mfc_list_merra2_polar'].values.tolist()
mfc_max_list_merra2_polar=ds['mfc_max_list_merra2_polar'].values.tolist()

ivt_array_merra2_fixed_poleward=ds['ivt_array_merra2_fixed_poleward']
ivt_max_array_merra2_fixed_poleward=ds['ivt_max_array_merra2_fixed_poleward']
iwv_array_merra2_fixed_poleward=ds['iwv_array_merra2_fixed_poleward']
iwv_max_array_merra2_fixed_poleward=ds['iwv_max_array_merra2_fixed_poleward']
s_array_merra2_fixed_poleward=ds['s_array_merra2_fixed_poleward']
s_max_array_merra2_fixed_poleward=ds['s_max_array_merra2_fixed_poleward']
mfc_array_merra2_fixed_poleward=ds['mfc_array_merra2_fixed_poleward']
mfc_max_array_merra2_fixed_poleward=ds['mfc_max_array_merra2_fixed_poleward']

ivt_list_merra2_fixed_poleward=ds['ivt_list_merra2_fixed_poleward'].values
ivt_max_list_merra2_fixed_poleward=ds['ivt_max_list_merra2_fixed_poleward'].values
iwv_list_merra2_fixed_poleward=ds['iwv_list_merra2_fixed_poleward'].values
iwv_max_list_merra2_fixed_poleward=ds['iwv_max_list_merra2_fixed_poleward'].values
s_list_merra2_fixed_poleward=ds['s_list_merra2_fixed_poleward'].values
s_max_list_merra2_fixed_poleward=ds['s_max_list_merra2_fixed_poleward'].values
mfc_list_merra2_fixed_poleward=ds['mfc_list_merra2_fixed_poleward'].values
mfc_max_list_merra2_fixed_poleward=ds['mfc_max_list_merra2_fixed_poleward'].values

ds=xr.open_dataset(dir_data+'ar_chars_synoptic_era5.nc')

ivt_array_era5_polar=ds['ivt_array_era5_polar'].values
ivt_max_array_era5_polar=ds['ivt_max_array_era5_polar'].values
iwv_array_era5_polar=ds['iwv_array_era5_polar'].values
iwv_max_array_era5_polar=ds['iwv_max_array_era5_polar'].values
s_array_era5_polar=ds['s_array_era5_polar'].values
s_max_array_era5_polar=ds['s_max_array_era5_polar'].values
mfc_array_era5_polar=ds['mfc_array_era5_polar'].values
mfc_max_array_era5_polar=ds['mfc_max_array_era5_polar'].values
#mfc_sum_array_era5_polar=ds['mfc_sum_array_era5_polar'].values

ivt_list_era5_polar=ds['ivt_list_era5_polar'].values.tolist()
ivt_max_list_era5_polar=ds['ivt_max_list_era5_polar'].values.tolist()
iwv_list_era5_polar=ds['iwv_list_era5_polar'].values.tolist()
iwv_max_list_era5_polar=ds['iwv_max_list_era5_polar'].values.tolist()
s_list_era5_polar=ds['s_list_era5_polar'].values.tolist()
s_max_list_era5_polar=ds['s_max_list_era5_polar'].values.tolist()
mfc_list_era5_polar=ds['mfc_list_era5_polar'].values.tolist()
mfc_max_list_era5_polar=ds['mfc_max_list_era5_polar'].values.tolist()
#mfc_sum_list_era5_polar=ds['mfc_sum_list_era5_polar'].values.tolist()

ivt_array_era5_fixed=ds['ivt_array_era5_fixed'].values
ivt_max_array_era5_fixed=ds['ivt_max_array_era5_fixed'].values
iwv_array_era5_fixed=ds['iwv_array_era5_fixed'].values
iwv_max_array_era5_fixed=ds['iwv_max_array_era5_fixed'].values
s_array_era5_fixed=ds['s_array_era5_fixed'].values
s_max_array_era5_fixed=ds['s_max_array_era5_fixed'].values
mfc_array_era5_fixed=ds['mfc_array_era5_fixed'].values
mfc_max_array_era5_fixed=ds['mfc_max_array_era5_fixed'].values
#mfc_sum_array_era5_fixed=ds['mfc_sum_array_era5_fixed'].values

ivt_array_era5_fixed_poleward=ds['ivt_array_era5_fixed_poleward'].values
ivt_max_array_era5_fixed_poleward=ds['ivt_max_array_era5_fixed_poleward'].values
iwv_array_era5_fixed_poleward=ds['iwv_array_era5_fixed_poleward'].values
iwv_max_array_era5_fixed_poleward=ds['iwv_max_array_era5_fixed_poleward'].values
s_array_era5_fixed_poleward=ds['s_array_era5_fixed_poleward'].values
s_max_array_era5_fixed_poleward=ds['s_max_array_era5_fixed_poleward'].values
mfc_array_era5_fixed_poleward=ds['mfc_array_era5_fixed_poleward'].values
mfc_max_array_era5_fixed_poleward=ds['mfc_max_array_era5_fixed_poleward'].values
#mfc_sum_array_era5_fixed_poleward=ds['mfc_sum_array_era5_fixed_poleward'].values

ivt_list_era5_fixed=ds['ivt_list_era5_fixed'].values.tolist()
ivt_max_list_era5_fixed=ds['ivt_max_list_era5_fixed'].values.tolist()
iwv_list_era5_fixed=ds['iwv_list_era5_fixed'].values.tolist()
iwv_max_list_era5_fixed=ds['iwv_max_list_era5_fixed'].values.tolist()
s_list_era5_fixed=ds['s_list_era5_fixed'].values.tolist()
s_max_list_era5_fixed=ds['s_max_list_era5_fixed'].values.tolist()
mfc_list_era5_fixed=ds['mfc_list_era5_fixed'].values.tolist()
mfc_max_list_era5_fixed=ds['mfc_max_list_era5_fixed'].values.tolist()
#mfc_sum_list_era5_fixed=ds['mfc_sum_list_era5_fixed'].values.tolist()

ivt_list_era5_fixed_poleward=ds['ivt_list_era5_fixed_poleward'].values.tolist()
ivt_max_list_era5_fixed_poleward=ds['ivt_max_list_era5_fixed_poleward'].values.tolist()
iwv_list_era5_fixed_poleward=ds['iwv_list_era5_fixed_poleward'].values.tolist()
iwv_max_list_era5_fixed_poleward=ds['iwv_max_list_era5_fixed_poleward'].values.tolist()
s_list_era5_fixed_poleward=ds['s_list_era5_fixed_poleward'].values.tolist()
s_max_list_era5_fixed_poleward=ds['s_max_list_era5_fixed_poleward'].values.tolist()
mfc_list_era5_fixed_poleward=ds['mfc_list_era5_fixed_poleward'].values.tolist()
mfc_max_list_era5_fixed_poleward=ds['mfc_max_list_era5_fixed_poleward'].values.tolist()
#mfc_sum_list_era5_fixed_poleward=ds['mfc_sum_list_era5_fixed_poleward'].values.tolist()


ds=xr.open_dataset(dir_data+'ar_chars_synoptic_jra55.nc')

ivt_array_jra55_polar=ds['ivt_array_jra55_polar'].values
ivt_max_array_jra55_polar=ds['ivt_max_array_jra55_polar'].values
iwv_array_jra55_polar=ds['iwv_array_jra55_polar'].values
iwv_max_array_jra55_polar=ds['iwv_max_array_jra55_polar'].values
s_array_jra55_polar=ds['s_array_jra55_polar'].values
s_max_array_jra55_polar=ds['s_max_array_jra55_polar'].values
#mfc_array_jra55_polar=ds['mfc_array_jra55_polar'].values
#mfc_max_array_jra55_polar=ds['mfc_max_array_jra55_polar'].values


ivt_list_jra55_polar=ds['ivt_list_jra55_polar'].values.tolist()
ivt_max_list_jra55_polar=ds['ivt_max_list_jra55_polar'].values.tolist()
iwv_list_jra55_polar=ds['iwv_list_jra55_polar'].values.tolist()
iwv_max_list_jra55_polar=ds['iwv_max_list_jra55_polar'].values.tolist()
s_list_jra55_polar=ds['s_list_jra55_polar'].values.tolist()
s_max_list_jra55_polar=ds['s_max_list_jra55_polar'].values.tolist()
#mfc_list_era5_polar=ds['mfc_list_era5_polar'].values.tolist()
#mfc_max_list_era5_polar=ds['mfc_max_list_era5_polar'].values.tolist()

ivt_array_jra55_fixed=ds['ivt_array_jra55_fixed'].values
ivt_max_array_jra55_fixed=ds['ivt_max_array_jra55_fixed'].values
iwv_array_jra55_fixed=ds['iwv_array_jra55_fixed'].values
iwv_max_array_jra55_fixed=ds['iwv_max_array_jra55_fixed'].values
s_array_jra55_fixed=ds['s_array_jra55_fixed'].values
s_max_array_jra55_fixed=ds['s_max_array_jra55_fixed'].values
#mfc_array_era5_fixed=ds['mfc_array_era5_fixed'].values
#mfc_max_array_era5_fixed=ds['mfc_max_array_era5_fixed'].values

ivt_array_jra55_fixed_poleward=ds['ivt_array_jra55_fixed_poleward'].values
ivt_max_array_jra55_fixed_poleward=ds['ivt_max_array_jra55_fixed_poleward'].values
iwv_array_jra55_fixed_poleward=ds['iwv_array_jra55_fixed_poleward'].values
iwv_max_array_jra55_fixed_poleward=ds['iwv_max_array_jra55_fixed_poleward'].values
s_array_jra55_fixed_poleward=ds['s_array_jra55_fixed_poleward'].values
s_max_array_jra55_fixed_poleward=ds['s_max_array_jra55_fixed_poleward'].values
#mfc_array_era5_fixed_poleward=ds['mfc_array_era5_fixed_poleward'].values
#mfc_max_array_era5_fixed_poleward=ds['mfc_max_array_era5_fixed_poleward'].values

ivt_list_jra55_fixed=ds['ivt_list_jra55_fixed'].values.tolist()
ivt_max_list_jra55_fixed=ds['ivt_max_list_jra55_fixed'].values.tolist()
iwv_list_jra55_fixed=ds['iwv_list_jra55_fixed'].values.tolist()
iwv_max_list_jra55_fixed=ds['iwv_max_list_jra55_fixed'].values.tolist()
s_list_jra55_fixed=ds['s_list_jra55_fixed'].values.tolist()
s_max_list_jra55_fixed=ds['s_max_list_jra55_fixed'].values.tolist()
#mfc_list_era5_fixed=ds['mfc_list_era5_fixed'].values.tolist()
#mfc_max_list_era5_fixed=ds['mfc_max_list_era5_fixed'].values.tolist()

ivt_list_jra55_fixed_poleward=ds['ivt_list_jra55_fixed_poleward'].values.tolist()
ivt_max_list_jra55_fixed_poleward=ds['ivt_max_list_jra55_fixed_poleward'].values.tolist()
iwv_list_jra55_fixed_poleward=ds['iwv_list_jra55_fixed_poleward'].values.tolist()
iwv_max_list_jra55_fixed_poleward=ds['iwv_max_list_jra55_fixed_poleward'].values.tolist()
s_list_jra55_fixed_poleward=ds['s_list_jra55_fixed_poleward'].values.tolist()
s_max_list_jra55_fixed_poleward=ds['s_max_list_jra55_fixed_poleward'].values.tolist()
#mfc_list_era5_fixed_poleward=ds['mfc_list_era5_fixed_poleward'].values.tolist()
#mfc_max_list_era5_fixed_poleward=ds['mfc_max_list_era5_fixed_poleward'].values.tolist()

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(ivt_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(ivt_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(ivt_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(ivt_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(ivt_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(ivt_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(ivt_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(ivt_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(ivt_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(ivt_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(ivt_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(ivt_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(ivt_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(ivt_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(ivt_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(ivt_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(ivt_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(ivt_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1. for x in ivt_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in ivt_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in ivt_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],350,520,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in ivt_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in ivt_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in ivt_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in ivt_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in ivt_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in ivt_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in ivt_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in ivt_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in ivt_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('kg m$^{-1}$ s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{IVT}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.5,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(350,520)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_1.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(ivt_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(ivt_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(ivt_max_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(ivt_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(ivt_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(ivt_max_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(ivt_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(ivt_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(ivt_max_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(ivt_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(ivt_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(ivt_max_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(ivt_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(ivt_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(ivt_max_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(ivt_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(ivt_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(ivt_max_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1. for x in ivt_max_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1. for x in ivt_max_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2023,1)
test=[x/1. for x in ivt_max_list_jra55_fixed[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_jra55_polar[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in ivt_max_list_jra55_fixed_poleward[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],651,1075,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in ivt_max_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in ivt_max_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in ivt_max_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in ivt_max_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in ivt_max_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in ivt_max_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in ivt_max_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in ivt_max_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in ivt_max_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=ivt_max_list_merra2_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_merra2_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_merra2_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=ivt_max_list_era5_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_era5_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_era5_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=ivt_max_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=ivt_max_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('kg m$^{-1}$ s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{max}$'+' '+r'$\bf{IVT}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.3,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(651,1075)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_2.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(iwv_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(iwv_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(iwv_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(iwv_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(iwv_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(iwv_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(iwv_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(iwv_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(iwv_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(iwv_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(iwv_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(iwv_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(iwv_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(iwv_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(iwv_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(iwv_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(iwv_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(iwv_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1. for x in iwv_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1. for x in iwv_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2023,1)
test=[x/1. for x in iwv_list_jra55_fixed[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_jra55_polar[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_list_jra55_fixed_poleward[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],23.05,32.3,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in iwv_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in iwv_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in iwv_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in iwv_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in iwv_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in iwv_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in iwv_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in iwv_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in iwv_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=iwv_list_merra2_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_merra2_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_merra2_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=iwv_list_era5_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_era5_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_era5_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=iwv_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('kg m$^{-2}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{IWV}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.8,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(23.05,32.3)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_3.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(iwv_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(iwv_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(iwv_max_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(iwv_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(iwv_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(iwv_max_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(iwv_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(iwv_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(iwv_max_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(iwv_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(iwv_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(iwv_max_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(iwv_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(iwv_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(iwv_max_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(iwv_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(iwv_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(iwv_max_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1. for x in iwv_max_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1. for x in iwv_max_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2023,1)
test=[x/1. for x in iwv_max_list_jra55_fixed[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_jra55_polar[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in iwv_max_list_jra55_fixed_poleward[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],35,51,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in iwv_max_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in iwv_max_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in iwv_max_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in iwv_max_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in iwv_max_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in iwv_max_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in iwv_max_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in iwv_max_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in iwv_max_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=iwv_max_list_merra2_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_merra2_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_merra2_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=iwv_max_list_era5_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_era5_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_era5_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=iwv_max_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=iwv_max_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('kg m$^{-2}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{max}$'+' '+r'$\bf{IWV}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.8,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(35,51)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_4.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(s_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(s_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(s_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(s_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(s_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(s_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(s_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(s_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(s_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(s_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(s_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(s_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(s_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(s_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(s_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(s_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(s_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(s_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1. for x in s_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1. for x in s_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2023,1)
test=[x/1. for x in s_list_jra55_fixed[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_jra55_polar[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_list_jra55_fixed_poleward[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],14,19.3,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in s_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in s_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in s_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in s_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in s_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in s_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in s_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=s_list_merra2_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_merra2_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_merra2_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=s_list_era5_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_era5_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_era5_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=s_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('m s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.2,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(14.01,19.3)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_5.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(s_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(s_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(s_max_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(s_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(s_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(s_max_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(s_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(s_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(s_max_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(s_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(s_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(s_max_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_jra55_polar=list(np.asarray(s_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=list(np.asarray(s_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=list(np.asarray(s_max_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_jra55_polar=ivt_month_jra55_polar+list(np.asarray(s_max_array_jra55_polar)[:,i-1])
        ivt_month_jra55_fixed=ivt_month_jra55_fixed+list(np.asarray(s_max_array_jra55_fixed)[:,i-1])
        ivt_month_jra55_fixed_poleward=ivt_month_jra55_fixed_poleward+list(np.asarray(s_max_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1. for x in s_max_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1. for x in s_max_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2023,1)
test=[x/1. for x in s_max_list_jra55_fixed[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_jra55_polar[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in s_max_list_jra55_fixed_poleward[22:-1]]
r=st.linregress(yrs2,test)
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,ivt_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,ivt_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,ivt_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,ivt_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],24,34.75,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in s_max_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in s_max_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in s_max_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in s_max_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in s_max_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in s_max_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in s_max_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in s_max_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in s_max_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=s_max_list_merra2_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_merra2_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_merra2_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=s_max_list_era5_polar[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_era5_fixed[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_era5_fixed_poleward[-23:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=s_max_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=s_max_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('m s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{max}$'+' '+r'$\bf{850}$'+' '+r'$\bf{hPa}$'+' '+r'$\bf{wind}$'+' '+r'$\bf{speed}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.2,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(24.01,34.75)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_6.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(mfc_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(mfc_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(mfc_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(mfc_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(mfc_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(mfc_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(mfc_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(mfc_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(mfc_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(mfc_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(mfc_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(mfc_array_merra2_fixed_poleward)[:,i-1])
        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1e-5 for x in mfc_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1e-5 for x in mfc_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_polar],color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_fixed],color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_fixed_poleward],color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_polar],color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_fixed],color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_fixed_poleward],color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],4.8,11.1,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1e-5 for x in mfc_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1e-5 for x in mfc_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1e-5 for x in mfc_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1e-5 for x in mfc_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1e-5 for x in mfc_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1e-5 for x in mfc_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=[x/1e-5 for x in mfc_list_merra2_polar[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_list_merra2_fixed[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_list_merra2_fixed_poleward[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=[x/1e-5 for x in mfc_list_era5_polar[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_list_era5_fixed[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_list_era5_fixed_poleward[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('10$^{-5}$ kg m$^{-2}$ s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{MFC}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.9,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(4.8,11.1)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_7.png')

In [ ]:
yrs=np.arange(1940,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_era5_polar=list(np.asarray(mfc_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=list(np.asarray(mfc_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=list(np.asarray(mfc_max_array_era5_fixed_poleward)[:,i])
        
    else:
        ivt_month_era5_polar=ivt_month_era5_polar+list(np.asarray(mfc_max_array_era5_polar)[:,i])
        ivt_month_era5_fixed=ivt_month_era5_fixed+list(np.asarray(mfc_max_array_era5_fixed)[:,i])
        ivt_month_era5_fixed_poleward=ivt_month_era5_fixed_poleward+list(np.asarray(mfc_max_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2023,1)
for i in range(len(yrs)):
    if i==0:
        ivt_month_merra2_polar=list(np.asarray(mfc_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=list(np.asarray(mfc_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=list(np.asarray(mfc_max_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        ivt_month_merra2_polar=ivt_month_merra2_polar+list(np.asarray(mfc_max_array_merra2_polar)[:,i-1])
        ivt_month_merra2_fixed=ivt_month_merra2_fixed+list(np.asarray(mfc_max_array_merra2_fixed)[:,i-1])
        ivt_month_merra2_fixed_poleward=ivt_month_merra2_fixed_poleward+list(np.asarray(mfc_max_array_merra2_fixed_poleward)[:,i-1])
        
x_arr=np.arange(1979.5,2022.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2023,1)
test=[x/1e-5 for x in mfc_max_list_merra2_fixed]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_max_list_merra2_polar]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_max_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2023,1)
test=[x/1e-5 for x in mfc_max_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_max_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e-5 for x in mfc_max_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940,2023,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_polar],color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_fixed],color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_merra2_fixed_poleward],color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2022.5,1./12.)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_polar],color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_fixed],color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e-5 for x in ivt_month_era5_fixed_poleward],color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1978.5],172,295,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1e-5 for x in mfc_max_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1e-5 for x in mfc_max_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1e-5 for x in mfc_max_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1e-5 for x in mfc_max_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1e-5 for x in mfc_max_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
#ax.plot(yrs2,[x/1. for x in s_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1e-5 for x in mfc_max_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=1.5
yrs=np.arange(2000,2023,1)
test=[x/1e-5 for x in mfc_max_list_merra2_polar[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_max_list_merra2_fixed[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_max_list_merra2_fixed_poleward[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2023,1)
test=[x/1e-5 for x in mfc_max_list_era5_polar[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_max_list_era5_fixed[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e-5 for x in mfc_max_list_era5_fixed_poleward[-23:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('10$^{-5}$ kg m$^{-2}$ s$^{-1}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{max}$'+' '+r'$\bf{MFC}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.0,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(172,295)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_synoptic_8.png')